Notwenige Importe

In [1]:
import sys
sys.path.append("src")  # Falls noch nicht gesetzt

from audio_data import AudioDataSet
from feature_set import FeatureSet
from classifier import DroneClassifier

Konfigurationsfile

In [2]:
config = {
    "sample_rate": 16000,
    "audio_length": 1, # in Sekunden
    "train_path": "d:/Dropbox/03 H2 Think/AuDroK mFund/Auswertungen/Datensätze/Drone vs. No Drone/TRAINING/",
    "val_path":   "d:/Dropbox/03 H2 Think/AuDroK mFund/Auswertungen/Datensätze/Drone vs. No Drone/VALIDATION/",
    "model_file": "models/classifier_001.keras",
    "feature_files": {
        "train_features": "models/train_features.pkl",
        "val_features": "models/val_features.pkl",
        "train_labels": "models/train_labels.pkl",
        "val_labels": "models/val_labels.pkl"
    }
}

Rohdaten fürs Training laden und inspizieren

In [3]:
raw_data = AudioDataSet.from_path(
    path=config["train_path"],
    sample_rate=config["sample_rate"]
)

Processing audio files (3267 files to process):
[#######################################.] 99.45%


Apply Pre-Emphasis auf die Trainingsrohdaten

In [4]:
for audio in raw_data.audio:
    audio.apply_pre_emphasis()

Chunken der Trainingsrohdaten

In [5]:
chunk_length = config["sample_rate"] * config["audio_length"]
chunked_data = raw_data.chunk_all(chunk_length)
print(f"{len(chunked_data.audio)} Chunks")

39171 Chunks


Zwischenspeichern der Trainingsdaten zur Vereinfachung des Debuggings (kann später wieder entfernt werden)

In [6]:
# Speichern
import pickle

with open("chunked_data.pkl", "wb") as f:
    pickle.dump(chunked_data, f)

In [ ]:
# Laden
import pickle

with open("chunked_data.pkl", "rb") as f:
    chunked_data = pickle.load(f)

Augmentieren der Chunks durch das Hinzufügen von Bodenreflexionen

In [7]:
from augmentations import apply_ground_reflection_to_dataset

ranges = {
    "src_x": (-10, 10),
    "src_y": (-10, 10),
    "src_z": (1, 5),
    "mic_z": (1, 2),
    "attenuation": (0.05, 0.95)
}

augmented_data = apply_ground_reflection_to_dataset(
    dataset=chunked_data,
    sample_rate=config["sample_rate"],
    ranges=ranges
)

Rohdaten für die Validierung laden

In [8]:
raw_data_val = AudioDataSet.from_path(
    path=config["val_path"],
    sample_rate=config["sample_rate"]
)

Processing audio files (277 files to process):
[####################################....] 92.42%


Apply Pre-Emphasis auf die Validierungsdaten

In [9]:
for audio in raw_data_val.audio:
    audio.apply_pre_emphasis()

Chunken der Validierungsdaten

In [10]:
chunk_length = config["sample_rate"] * config["audio_length"]
chunked_data_val = raw_data_val.chunk_all(chunk_length)
print(f"{len(chunked_data_val.audio)} Chunks")

34211 Chunks


Zwischenspeichern der Validierungsdaten zur Vereinfachung des Debuggings (kann später wieder entfernt werden)

In [12]:
# Speichern
import pickle

with open("chunked_data_val.pkl", "wb") as f:
    pickle.dump(chunked_data_val, f)

In [ ]:
# Laden
import pickle

with open("chunked_data_val.pkl", "rb") as f:
    chunked_data_val = pickle.load(f)

Feature Extraction

In [11]:
# Feature-Extraktion
train_features = FeatureSet(augmented_data, sample_rate=config["sample_rate"])
X_train, y_train = train_features.extract()

val_features = FeatureSet(chunked_data_val, sample_rate=config["sample_rate"])
X_val, y_val = val_features.extract()

Processing features (39171 files to process):
[########################################] 100.00%
Processing features (34211 files to process):
[########################################] 100.00%


Modelltraining & Evaluation

In [15]:
model = DroneClassifier(model_path="models/classifier_002.h5")
model.train(X_train, y_train, X_val, y_val)
accuracy = model.evaluate(X_val, y_val)

print(f"Validierungsgenauigkeit: {accuracy:.2%}")

Encoded Labels - Train: [0 1]
Encoded Labels - Validation: [0 1]
Class weights: {0: np.float64(0.559138403562864), 1: np.float64(4.727371469949312)}
Epoch 1/30
613/613 ━━━━━━━━━━━━━━━━━━━━ 732s 1s/step - accuracy: 0.8509 - loss: 1.8983 - val_accuracy: 0.4775 - val_loss: 0.8862 - learning_rate: 5.0000e-04
Epoch 2/30
613/613 ━━━━━━━━━━━━━━━━━━━━ 662s 1s/step - accuracy: 0.9882 - loss: 0.0445 - val_accuracy: 0.6862 - val_loss: 0.2355 - learning_rate: 5.0000e-04
Epoch 3/30
613/613 ━━━━━━━━━━━━━━━━━━━━ 655s 1s/step - accuracy: 0.9884 - loss: 0.0617 - val_accuracy: 0.3554 - val_loss: 1.5770 - learning_rate: 5.0000e-04
Epoch 4/30
613/613 ━━━━━━━━━━━━━━━━━━━━ 636s 1s/step - accuracy: 0.9883 - loss: 0.2054 - val_accuracy: 0.5701 - val_loss: 0.3839 - learning_rate: 5.0000e-04
Epoch 5/30
613/613 ━━━━━━━━━━━━━━━━━━━━ 645s 1s/step - accuracy: 0.9923 - loss: 0.0578 - val_accuracy: 0.6022 - val_loss: 0.1819 - learning_rate: 5.0000e-04
Epoch 6/30
613/613 ━━━━━━━━━━━━━━━━━━━━ 640s 1s/step - accuracy: 0

1070/1070 ━━━━━━━━━━━━━━━━━━━━ 151s 141ms/step


ValueError: Mix of label input types (string and number)